# Q1.Part-1: MATCH AND MISMATCH VOCABULARY EXPLORATION

In [1]:
# DEFINING THE SPARK SESSION AND IMPORTING PACKAGES

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType,IntegerType, FloatType
from pyspark.sql.functions import concat,lit
import csv
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string


# INITIATE THE SPARK SESSION
spark = SparkSession \
    .builder \
    .appName("Python Spark Q1") \
    .getOrCreate()

# DEFINE FILE PATHS
dev_matched = "s3://comp5349-snaz3253/Assignment_Data/dev_matched.tsv"
test_matched = "s3://comp5349-snaz3253/Assignment_Data/test_matched.tsv"
dev_mis_matched = "s3://comp5349-snaz3253/Assignment_Data/dev_mismatched.tsv"
test_mis_matched = "s3://comp5349-snaz3253/Assignment_Data/test_mismatched.tsv"


# IMPORTING DATA INTO DATAFRAMES
dev_match_df = spark.read.csv(dev_matched,sep='\t',header=True,inferSchema="true")
test_match_df = spark.read.csv(test_matched,sep='\t',header=True,inferSchema="true")
dev_mis_match_df = spark.read.csv(dev_mis_matched,sep='\t',header=True,inferSchema="true")
test_mis_match_df = spark.read.csv(test_mis_matched,sep='\t',header=True,inferSchema="true")

# EXTRACTING USEFUL COLUMNS FROM DATAFRAMES
dev_match_df = dev_match_df.select(['sentence1','sentence2'])
test_match_df = test_match_df.select(['sentence1','sentence2'])
dev_mis_match_df = dev_mis_match_df.select(['sentence1','sentence2'])
test_mis_match_df = test_mis_match_df.select(['sentence1','sentence2'])

# MERGING DATAFRAMES TO CREATE COMPLETE MATCH AND MISMATCH SETS
matched = dev_match_df.union(test_match_df)
mis_matched = dev_mis_match_df.union(test_mis_match_df)

# CONCATENATING THE TWO SENTENCES TO FORM A SINGLE SET
match_df = matched.select(concat(matched.sentence1, lit(" "), matched.sentence2).alias('joined'))
mis_matched_df = mis_matched.select(concat(mis_matched.sentence1, lit(" "), mis_matched.sentence2))


# CONVERTING TO RDDS NOW
matched_rdd = match_df.rdd.map(list)
mis_matched_rdd = mis_matched_df.rdd.map(list)


# DEFINING MAPPER FUNCTIONS FOR PERFORMING TRANSFORMATIONS

def punctuation_remover(record):
    list_punct=list(string.punctuation)
    x = str(record)
    for punct in list_punct:
        if punct in x:
            x = x.replace(punct, '')
    return (x)

def custom_tokenizer(string):
    words = word_tokenize(string)
    list_of_words = [(word.lower(),1) for word in words if word.isalpha()]
    return list_of_words

def pairFlagToWord(record, flag):
    return (record[0], flag)


# TRANSFORMING RDDS NOW

## Remove punctuations from sentences
matched_rdd_punct = matched_rdd.map(punctuation_remover)
mis_matched_rdd_punct = mis_matched_rdd.map(punctuation_remover)

## Extract tokens from sentences
matched_rdd_tokenized = matched_rdd_punct.flatMap(custom_tokenizer)
mis_matched_rdd_tokenized = mis_matched_rdd_punct.flatMap(custom_tokenizer)

## Aggregate the tokens to give unqiue tokens and their number
matched_unique_tokens = matched_rdd_tokenized.aggregateByKey(0, (lambda x, y : x + y), (lambda x, y : x + y), 1)
mis_matched_unique_tokens = mis_matched_rdd_tokenized.aggregateByKey(0, (lambda x, y : x + y), (lambda x, y : x + y), 1)

## Flagging the words to show which set of vocabulary they belong to
flagged_match_tokens = matched_unique_tokens.map(lambda row: pairFlagToWord(row,"match_vocabulary"))
flagged_mismatch_tokens = mis_matched_unique_tokens.map(lambda row: pairFlagToWord(row,"mismatch_vocabulary"))

# COUNTING COMMON AND UNIQUE WORDS 

## Join the entire match and mismatch corpuses
Complete_Corpus = flagged_match_tokens.fullOuterJoin(flagged_mismatch_tokens)
Complete_Corpus.cache()

## Convert flagged df to dataframe
Complete_Corpus_df = spark.createDataFrame(Complete_Corpus).withColumnRenamed('_1','Word').withColumnRenamed('_2','Required_Combination')

#print out required statistics
print("The required statistics for Question 1 for the matched and mismatched corpora are as below: \n")
print('Total unique words in the combined corpus is:', Complete_Corpus_df.count())


print('\n\n***************************************************************************\n\n')

print('The various combinations of words across match and mismatch vocabulary sets are given as under:')
Complete_Corpus_df.groupBy('Required_Combination').count().show(truncate=False)


VBox()

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.
